In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/Users/munjismac/code/munjik/personal-projects/real_estate_project/RawData.csv')

In [3]:
df.head(10)

,CASE #,Special Master,Lender,Debtor,Additional Lien,Date,Time,Address,City,State,Zip,Balance,ARV,EQUITY,Sold,Postponed
0,D-202-CV-2019-00076,Robert Doyle,Loandepot.com,"Altamirano, Dominic; Altamirano, Julia",Cavalry SPV I; NMTR,1/5/21,10:15 AM,10528 Duke Ave SW,Albuquerque,NM,87121,"$97,318.40","$145,000.00","$47,681.60",NaN,CANCELLED
1,D-202-CV-2018-03849,Robert Doyle,US Bank,"Conlee, Nicole",HUD,1/5/21,10:15 AM,4605 Palo Duro Ave NE,Albuquerque,NM,87110,"$95,564.54","$200,700.00","$105,135.46",NaN,3/2/21
2,D-202-CV-2016-07895,Robert Doyle,Pingora Loan Servicing,"Estate of Gallegos, Steven G; Gallegos, Doreen...",Time Investment Co,1/5/21,10:15 AM,2636 Don Felipe Pl SW,Albuquerque,NM,87105,"$410,199.42","$361,000.00","($49,199.42)",273163,NaN
3,D-202-CV-2016-04200,Robert Doyle,US Bank,"Estate of Ulibarri, Gladys ; Ulibarri, Joseph ...",NaN,1/5/21,10:15 AM,7601 Winter Ave NE,Albuquerque,NM,87110,"$336,375.47","$307,000.00","($29,375.47)",NaN,2/2/21
4,D-202-CV-2018-04165,Robert Doyle,US Bank,"Garcia, Jamie S; Garcia, Miramon B",NMMFA,1/5/21,10:15 AM,413 Dolores Dr SW,Albuquerque,NM,87121,"$118,588.31","$149,000.00","$30,411.69",117450,NaN
5,D-202-CV-2015-04764,Robert Doyle,US Bank,"Giles, Larry G",NaN,1/5/21,10:15 AM,8123 Gem Pointe Rd SW,Albuquerque,NM,87121,"$168,188.76","$185,000.00","$16,811.24",180380.1,NaN
6,D-202-CV-2019-05980,Robert Doyle,"BOKF, NA","Kang, Joo E; Kang, Helen",NaN,1/5/21,10:15 AM,5008 General Bradley St NE,Albuquerque,NM,87111,"$270,255.41","$311,310.00","$41,054.59",NaN,3/2/21
7,D-202-CV-2017-02886,Robert Doyle,"BOKF, NA","Martinez, Betty J; Everly, Rebecca",NMTR,1/5/21,10:15 AM,2101 Spring Canyon Dr SW,Albuquerque,NM,87121,"$85,326.64","$175,000.00","$89,673.36",NaN,2/2/21
8,D-202-CV-2018-03958,Robert Doyle,US Bank,"Price, Timothy A",HUD; NMMFA; NMTR; MVD; LVNV Funding,1/5/21,10:15 AM,227 Hwy 217,Tijeras,NM,87059,"$120,141.43",NaN,NaN,NaN,NaN
9,D-202-CV-2018-05154,Robert Doyle,US Bank,"Smith, Doretha A",Aqua Finance; Citibank SD; NMTR,1/5/21,10:15 AM,9309 Halyard Rd NW,Albuquerque,NM,87121,"$171,443.30","$178,000.00","$6,556.70",108500,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CASE #           126 non-null    object
 1   Special Master   128 non-null    object
 2   Lender           128 non-null    object
 3   Debtor           128 non-null    object
 4   Additional Lien  96 non-null     object
 5   Date             128 non-null    object
 6   Time             128 non-null    object
 7   Address          128 non-null    object
 8   City             128 non-null    object
 9   State            128 non-null    object
 10  Zip              128 non-null    object
 11  Balance          128 non-null    object
 12  ARV              124 non-null    object
 13  EQUITY           123 non-null    object
 14  Sold             36 non-null     object
 15  Postponed        79 non-null     object
dtypes: object(16)
memory usage: 16.2+ KB


<h2>Let's clean up some data </h2> 

In [5]:
# remove the unwanted columns
df = df.drop(columns=['CASE #', 'Special Master','Additional Lien','Postponed', 'Time', 'Postponed','Debtor'])

In [6]:
df = df.drop(labels=128) #last column was all empty anyways
df.isnull().sum() #check to see how many null values we have in each column

Lender      0
Date        0
Address     0
City        0
State       0
Zip         0
Balance     0
ARV         4
EQUITY      5
Sold       92
dtype: int64

In [7]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="most_frequent") # Instanciate a SimpleImputer object with strategy of choice
# impute lender
imputer.fit(df[['ARV']]) # Call the "fit" method on the object
df['ARV'] = imputer.transform(df[['ARV']]) # Call the "transform" method on the object

imputer.fit(df[['EQUITY']]) # Call the "fit" method on the object
df['EQUITY'] = imputer.transform(df[['EQUITY']]) # Call the "transform" method on the object


In [8]:
df.head(10)

,Lender,Date,Address,City,State,Zip,Balance,ARV,EQUITY,Sold
0,Loandepot.com,1/5/21,10528 Duke Ave SW,Albuquerque,NM,87121,"$97,318.40","$145,000.00","$47,681.60",NaN
1,US Bank,1/5/21,4605 Palo Duro Ave NE,Albuquerque,NM,87110,"$95,564.54","$200,700.00","$105,135.46",NaN
2,Pingora Loan Servicing,1/5/21,2636 Don Felipe Pl SW,Albuquerque,NM,87105,"$410,199.42","$361,000.00","($49,199.42)",273163
3,US Bank,1/5/21,7601 Winter Ave NE,Albuquerque,NM,87110,"$336,375.47","$307,000.00","($29,375.47)",NaN
4,US Bank,1/5/21,413 Dolores Dr SW,Albuquerque,NM,87121,"$118,588.31","$149,000.00","$30,411.69",117450
5,US Bank,1/5/21,8123 Gem Pointe Rd SW,Albuquerque,NM,87121,"$168,188.76","$185,000.00","$16,811.24",180380.1
6,"BOKF, NA",1/5/21,5008 General Bradley St NE,Albuquerque,NM,87111,"$270,255.41","$311,310.00","$41,054.59",NaN
7,"BOKF, NA",1/5/21,2101 Spring Canyon Dr SW,Albuquerque,NM,87121,"$85,326.64","$175,000.00","$89,673.36",NaN
8,US Bank,1/5/21,227 Hwy 217,Tijeras,NM,87059,"$120,141.43","$190,000.00","$117,907.81",NaN
9,US Bank,1/5/21,9309 Halyard Rd NW,Albuquerque,NM,87121,"$171,443.30","$178,000.00","$6,556.70",108500


In [9]:
df.isnull().sum()

Lender      0
Date        0
Address     0
City        0
State       0
Zip         0
Balance     0
ARV         0
EQUITY      0
Sold       92
dtype: int64

In [11]:
# function to remove puncuation
def remove_dollar(x):
    return x.replace('$','')
def replace_comma(x):
    return x.replace(',','')
def replace_period(x):
    return x.replace('.','')
def replace_paran(x):
    return x.replace("(","").replace(")","")
# reformat from str to float
def currency_to_float(x: str) -> float:
    """Converts US currency ``x`` to float."""
    return float(x)
    
#-> potentially use this function to do all in one functions here
# Convert the currency to float.
# def currency_to_float(x: str) -> float:
#     """Converts US currency ``x`` to float."""
#     return float(x.replace("$", "").replace(",", ""))

In [12]:
df['Sold'] = df['Sold'].astype('str')
df['Sold'] = df['Sold'].apply(remove_dollar)
df['Sold'] = df['Sold'].apply(replace_comma)

In [24]:
df[(df.Sold) == 'THEY DONT HAVE ANYTHING ON THIS...he\'ll double check when back at office']
df.iloc[63]['Sold'] = 'NaN'
df['Sold'] = df['Sold'].apply(lambda x: currency_to_float(x))


,Lender,Date,Address,City,State,Zip,Balance,ARV,EQUITY,Sold
63,Deutsche Bank,2/4/21,3406 Pickard Ave NE,Albuquerque,NM,87110,"$264,350.17","$333,000.00","$68,649.83",THEY DONT HAVE ANYTHING ON THIS...he'll double...


In [35]:
# sum should have increased from 92 to 93
df['Sold'].isnull().sum()

93

In [36]:
mean_imputer = SimpleImputer(strategy="mean") # Instanciate a SimpleImputer object with strategy of choice
mean_imputer.fit(df[['Sold']]) # Call the "fit" method on the object
df['Sold'] = mean_imputer.transform(df[['Sold']]) # Call the "transform" method on the object

In [53]:
df['Sold'].isnull().sum() #now should be zero once we fill the values in with imputes

0

In [38]:
# First convert to type string -> clean Balance column
df['Balance'] = df['Balance'].astype('str')
df['Balance'] = df['Balance'].apply(replace_comma)
df['Balance'] = df['Balance'].apply(remove_dollar)

# First convert to type string -> clean ARV column
df['ARV'] = df['ARV'].astype('str')
df['ARV'] = df['ARV'].apply(replace_comma)
df['ARV'] = df['ARV'].apply(remove_dollar)
# First convert to type string -> clean Equity column
df['EQUITY'] = df['EQUITY'].astype('str')
df['EQUITY'] = df['EQUITY'].apply(replace_comma)
df['EQUITY'] = df['EQUITY'].apply(remove_dollar)
df['EQUITY'] = df['EQUITY'].apply(replace_paran)

In [66]:
# FIXING THIS ERROR: ValueError: could not convert string to float: 'COMMERCIAL'
df[(df.ARV) == 'COMMERCIAL'] #find location
df.loc[df.ARV == "COMMERCIAL", "ARV"] = None

In [70]:
df.isnull().sum() #sum for ARV has gone up to 1 because of code above
mean_imputer.fit(df[['ARV']]) # Call the "fit" method on the object
df['ARV'] = mean_imputer.transform(df[['ARV']]) # Call the "transform" method on the object

In [73]:
mean_imputer.statistics_

array([221713.1496063])

In [75]:
df['Balance'] = df['Balance'].apply(lambda x: currency_to_float(x))
df['ARV'] = df['ARV'].apply(lambda x: currency_to_float(x))
df['EQUITY'] = df['EQUITY'].apply(lambda x: currency_to_float(x))

In [82]:
# lets convert Date to datetime64
df['Date'] = pd.to_datetime(df['Date'])

In [85]:
df

,Lender,Date,Address,City,State,Zip,Balance,ARV,EQUITY,Sold
0,Loandepot.com,2021-01-05,10528 Duke Ave SW,Albuquerque,NM,87121,97318.40,145000.0,47681.60,158512.216286
1,US Bank,2021-01-05,4605 Palo Duro Ave NE,Albuquerque,NM,87110,95564.54,200700.0,105135.46,158512.216286
2,Pingora Loan Servicing,2021-01-05,2636 Don Felipe Pl SW,Albuquerque,NM,87105,410199.42,361000.0,49199.42,273163.000000
3,US Bank,2021-01-05,7601 Winter Ave NE,Albuquerque,NM,87110,336375.47,307000.0,29375.47,158512.216286
4,US Bank,2021-01-05,413 Dolores Dr SW,Albuquerque,NM,87121,118588.31,149000.0,30411.69,117450.000000
...,...,...,...,...,...,...,...,...,...,...
123,Wells Fargo Bank,2021-03-23,247 Espanola NE,Albuquerque,NM,87108,81796.17,136000.0,54203.83,158512.216286
124,Deutsche Bank,2021-03-25,627 Cebolleta Ct SW,Albuquerque,NM,87105,160369.83,175000.0,14630.17,158512.216286
125,Rushmore Loan Management Services,2021-03-26,117 Sedilo Rd,Tijeras,NM,87059,189679.29,233000.0,43320.71,187301.000000
126,Freedom Mortgage Corp,2021-03-26,1301 Summerfield Pl SW,Albuquerque,NM,87121,154231.50,180000.0,25768.50,170600.000000


In [113]:
"""FOR REFERENCE ->
One Hot Encoding:
We use this categorical data encoding technique when the features are nominal(do not have any order)
------------------------------------------------------------------------------------------------
Label Encoding or Ordinal Encoding:
We use this categorical data encoding technique when the categorical feature is ordinal. In this case,
retaining the order is important. Hence encoding should reflect the sequence.
"""

'FOR REFERENCE ->\nOne Hot Encoding:\nWe use this categorical data encoding technique when the features are nominal(do not have any order)\n------------------------------------------------------------------------------------------------\nLabel Encoding or Ordinal Encoding:\nWe use this categorical data encoding technique when the categorical feature is ordinal. In this case,\nretaining the order is important. Hence encoding should reflect the sequence.\n'

In [109]:
print(df.Lender.nunique()) #OHE this or see next best step for 33 diff lenders
print(df.Address.nunique()) #--> I'd like to change this to Lat's and Longs for ML model
print(df.City.nunique()) #Encode this ? I think
print(df.State.nunique()) #easy encode all 1's
print(df.Zip.nunique())

33
91
2
1
17


In [ ]:
""""""

In [ ]:
# starting off by starting to see what is selling above judgement and %%% sold vs Scheduled 

In [ ]:
#1.)if sold is above balanced 
#2.) change the objects to their types
#3.) NaN == 0
#4.) Banks to individual #'s' 1 - 33 top 10 banks
# data[(data.Medal == 'Silver') & (data.Gender == 'Men')]



In [ ]:
# sns.heatmap(df)